# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['EndpointW',
 'FilterInterface',
 'FilterModeW',
 'PacketSettingsW',
 'PacketW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'new_transport_interface']

## Transport layer setup

Create the Transport layer and run it in the background.

In [3]:
tiface = await new_transport_interface("0.0.0.0", 12345)
tiface.run()

<Future pending cb=[<builtins.PyDoneCallback object at 0x7f536e47b610>()]>

# Filter layer setup

In [4]:
fiface = FilterInterface(tiface, FilterModeW("client"))

Find methods to run

In [5]:
[m for m in dir(fiface) if not m.startswith('__')]

['run']

Run Filter!

In [6]:
run_fut = fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not run_fut.done()